# Learning goals
After this week's lesson you should be able to:
- Refresher on data structure (DF vs Series vs List vs Array)
- Checking a columns data types and converting types
- Rename a (geo)dataframe column 
- parsing dates
- slicing strings
- handling missing data. 
    - filtering out missing data
    - replacing missing data with the mean
- Merging 
- More on groupby-and-summarize
- (Defining and using a function)
- (iterating over rows)
- (applying a function)

This week's lessons are adapted from:
- [PPD599: Advanced Urban Analytics](https://github.com/gboeing/ppd599/tree/main/syllabus)
- [Geo-Python Lesson 5](https://geo-python-site.readthedocs.io/en/latest/notebooks/L5/processing-data-with-pandas.html)

In [ ]:
# We are going to start importing the libraries we need
# all in one cell. 
# It is a good practice to keep all the imports in one cell so that
# we can easily see what libraries we are using in the notebook.
import pandas as pd
import numpy as np
import geopandas as gpd

# 0. Refresher
Before we move on, let's go over the different types of data structures we've encountered so far. We are going to cover: 
- Pandas DataFrames
- Pandas Series
- Lists
- Arrays

## 0.1 Pandas: Dataframes

In [ ]:
msa = pd.read_csv('msa_by_pop.csv')

What makes `msa` a dataframe? 

In [ ]:
msa.head()

In [ ]:
# We can use the type() function to see what type of object we have
type(msa)

Now let's select just the `population_2020` column from `msa`. What makes the following a pandas Series? 

## 0.2 Pandas: Series

In [ ]:
msa['population_2020'].head()

In [ ]:
type(msa['population_2020'])

In [ ]:
population = pd.Series([19768458, 12997353, 9509934, 7759615, 7206841],    
            index=['NYC', 'LA', 'Chicago', 'Dallas', 'Houston'])
population


In [ ]:
type(population)

## 0.3 Python data structures: Lists

Now, let's just select the values from `population` as a **list**. 

In [ ]:
## This is a list of numbers
## A list is a collection of objects of any type.
## It is created by putting the objects in square brackets

list1 = [19768458, 12997353, 9509934, 7759615, 7206841]

In [ ]:
type(list1)

In [ ]:
list.mean()

In [ ]:
## This is also a list
## But instead of being a list of numbers, it is a list of strings

list2 = ['NYC', 'LA', 'Chicago', 'Dallas', 'Houston']

## 0.4 Numpy: Arrays

We are not going to cover numpy directly in the class. But there is another list-like data structure used by the `numpy` library called an **array**. 

Though we are inputting a list above to create a pandas Series, pandas will turn this list into a **numpy array**. All pandas series are basically just generalized version of numpy arrays. 

An array is a collection of objects of the same type. It is created by putting the objects in square brackets and using the `np.array()` function. A numpy array is most used for numerical calculations such as finding the mean, min, sum of a set of values. 

In [ ]:
list1_as_array = np.array(list1)

In [ ]:
list1_as_array

In [ ]:
list1_as_array.mean()

# 1. Data cleaning
As you might have already seen, when we work with data, it is not always in a shape that we can use it, sometimes column names are misspelled, there are missing values. You may also have noticed that often we can extract information from columns that might make them easier to work with. All these steps can be considered part of a data cleaning or data wrangling process, where we get the dataset ready to be used more effectively for our analysis purposes. 




## 1.1 Getting the data
Let's say we want to compare the relationship between the **total number of students in a general ed public school** to the **money spent on new school construction and improvements in that school**. 

### School Construction Authority

First, go ahead and download the [Active Projects Under Construction](https://data.cityofnewyork.us/Housing-Development/Active-Projects-Under-Construction/8586-3zfm) dataset as a CSV and save it down to the folder where this notebook is. This is a dataset of new school projects (Capacity) and Capital Improvement Projects (CIP) currently under Construction, created by the School Construction Authority. 




In [ ]:
## Here we are going to read a csv directly from the web
## We are going to use the read_csv() function from the pandas library
## 

projects_under_const = pd.read_csv('Active_Projects_Under_Construction.csv')

Also, go ahead and download the data dictionary `SCA Active Projects in Construction Data Dictionary.xlsx`. Data dictionaries often have explanations for what each column name represents and other useful information about the data. 


If you open up the data dictionary, does it correspond to the "Columns in this Dataset" section in the NYC OpenData's page on this dataset? No, right? We have to be careful about these inconsistencies, even in official portals.

Taking a look at the first five rows we can already see there is a lot of missing data in this dataset. 

In [ ]:
projects_under_const.head()

### Class size dataset
Also download the [2021 - 2022 Average Class Size by School](https://data.cityofnewyork.us/Education/2021-2022-Average-Class-Size-by-School/sgr7-hhwp) dataset, along with it's attachments. (Here, only `2021-2022 Average Class Size By School DD.xlsx` is the data dictoinary, the other is the dataset as an excel spreadsheet). 


In [ ]:
class_size = pd.read_csv('2021_-_2022_Average_Class_Size_by_School.csv')

In [ ]:
class_size.head()

Here, most of the columns make sense to me. From the data dictionary, I can see that Program Type is coded as follows:

- General Education (Gen Ed), 
- Integrated Co-Teaching (ICT), 
- Gifted and Talented (G&T), 
- Self-Contained (SC)
- Accelerated (Acc)"


What does not make sense is the `Minimum Class Size` column, which seems to be the same as the maximum class size column in some cases. Therefore, I'll likely not use this column.

## 1.2 Assessing Data Types
One of the next things we'll check is the data type for each column to make sure that they are in the right format. 

In [ ]:
class_size.dtypes

I would not necessarily change the data types for all columns (especially when there are a lot), but just the ones that you might potentially need. Here, `Maximum Class Size` is an `object` format (I'm going to ignore `Minimum Class Size` for now), likely because the size is sometimes input as `<INT` and sometimes `INT`. 



## 1.3 Removing strings

In order to change the data type of the min and max class size to an `int` we have to clean up those columns a little bit. 

The function `.replace('str_to_be_replaced','str_to_replace_with)` will take `str_to_be_replaced` and replace it with `str_to_replace_with`. Here, I'm setting `<` to be replaced by nothing which is expressed as an empty string `""`. 

In [ ]:
# Again, I'm going to check that the function worked as expected first
# here .str is a method that is applied to a string
# it is a vectorized string method
class_size['Maximum Class Size'].str.replace('<', "")

# (Vectorization is the process of converting an algorithm from operating on a single value at a time to operating on a set of values at one time
# but it's not super important for us to know what this is right now)

Now lets assign the result to a new column and why not rename the column to something in snake case. 


In [ ]:
class_size['max_class_size_clean'] = class_size['Maximum Class Size'].str.replace('<', "")

In [ ]:
class_size.head()

Now let's see if we can turn the column `min_class_size_clean` into an `integer`. 

In [ ]:
class_size['max_class_size_clean'].astype(int)

Oops, I guess we also have to replace the greater than `>`. 

In [ ]:
class_size['max_class_size_clean'] = class_size['max_class_size_clean'].str.replace('>', "")

## 1.4 Changing data types
Now let's try to change the data type for `max_class_size_clean`. 

`.astype()` changes your column types for a particular column. 


In [ ]:
## What I've done here is replace the old `max_class_size_clean` column with 
## a version of it that is an int
class_size['max_class_size_clean'] = class_size['max_class_size_clean'].astype(int)

In [ ]:
# Notice that `int` from above defaults to 64 bit integers. 
class_size['max_class_size_clean'].dtype

## 1.5 Slicing strings

### 1.5.1
The `projects_under_const` has a `Data as Of` column, which gives us some temporal variation in when, at least the data was added to the table. It could be useful, for instance, if we think that `Data as Of` is a rough proxy for when the project was funded or approved. 

In [ ]:
# Remember that NaN means "Not a Number".
# In other words, it is a missing value
projects_under_const['Data As Of'].head()

Let's say we want to extract year from these dates. We have another string-related function we can apply to all of our values under `Data As Of`. 

`.split()` splits strings around given separator/delimiter to create a list of strings. 

Here, we will use `/` as our separator. 

In [ ]:
projects_under_const['Data As Of'].str.split('/')

Now we just have to get the last value (where it exists) and create a new column with the year. 

In [ ]:
## [-1] is a way to access the last element of a list
projects_under_const['data_year'] = projects_under_const['Data As Of'].str.split('/').str[-1]

In [ ]:
# Notice that when there was an NaN, the split function returned a NaN
projects_under_const['data_year'].head()

### 1.5.2
We will eventually be comparing school attendance characteristics to money allocated through **merging along a common column name** at the **school level**.

What are out options here? Let's take  look. 

In [ ]:
projects_under_const.head()

In [ ]:
class_size.head()

Even though there is a **School Name** column in both datasets, the format seems to be quite different. 
- For the `projects_under_const` dataset, the school names are all over the place. Some are the name and borough separated by a `-`, some also include an `@` followed by a rough locationn. 
- For the `class_size` df, the school names are consistent, but we can see that it might be a pain to match the two. 

In [ ]:
projects_under_const['School Name']

In [ ]:
class_size['School Name']

Instead, I noticed that there's a `Building ID` column in the `projects_under_constr` DF (dataframe, for short) that, though is described unhelpfully as "ID of the Building" in the documentation, looks to be similar to the `DBN` from `class_size` DF. In fact, when I look at what `DBN` is in the class size documentation, it says that this column "Denotes cocatenation[sic] of district, borough and three digit school number."

I'm going to guess here that if I extract the "borough and three digit school number" part of `DBN`, this will match my `Building ID` column. 

Thankfully, it seems like there is a fixed number of characters I need extract from `DBN`: 
- Borough = 1
- School number = 3

In total, I will need the last 4 characters from `DBN`. We'll do this again with a string splice. 

In [ ]:
# Here I am going to use the str method to get the last 4 characters of the DBN
# within the square brackets, I am taking everything fourth from the end onwards
# That's what -4 means

class_size['DBN'].str[-4:]

Quick review of selecting ranges:

In [ ]:
# It's a little strange because backwards counting starts at -1

class_size['DBN'].str[-1:]

In [ ]:
## Here, 4: means that I want to start at the fifth character 
## because python starts counting at 0 for forward counting
class_size['DBN'].str[4:]


In [ ]:
## And if I wanted to select a slice of the string in the middle
## I can do the following
class_size['DBN'].str[1:4]

Back to our exericse, let's assign our slice to a new colunn called `bid`

In [ ]:
class_size['bid'] = class_size['DBN'].str[-4:]

In [ ]:
class_size.head()

## 1.6 Handling missing data
Now, let's say that our analysis depends knowing the year the data was created. There are a few ways of handling missing data. 

### 1.6.1 Removing rows 
We can remove those rows with data missing from a column that we are planning to use in our analysis. 

In [ ]:
# Here we are going to use the isna() function to check if the data_year column has a NaN
# isna() returns a boolean (True or False) for each row
# and we are going to use that boolean to filter the dataframe. 
# We are going to keep only the rows where the data_year column is not a NaN

projects_under_const_new = projects_under_const[projects_under_const['data_year'].isna()==False]

### 1.6.2 Replacing missing data
We can also replace the missing data with certain values: 
- We can replace the data with the mean of the non-NaN column values, for numerical values. (For instance, if our columns were something like "adult heights", then replacing the NaN with the mean values in the columns would allow us to leave the sample mean unchanged, which might be good for regression purposes). 
- We can also replace with the median (if you think there are outliers in the sample that might be skewing the mean)
- Replacing with the mode (most frequent value) would make more sense if we think that there's some default value 

**What would you do here?**

In [ ]:
# This gets the mode of the data_year column
mode_year = projects_under_const['data_year'].mode()


In [ ]:
# This fills the NaNs with the mode using the fillna() function
# fillna() is a method that fills in missing values with a value of your choice
projects_under_const['data_year'].fillna(mode_year)


In [ ]:
# Now write over the old data_year column with the new one
projects_under_const['data_year'] = projects_under_const['data_year'].fillna(mode_year)

## Q1. In-Class Exercise 1 (5 pts)
In the end, was it the best idea to replace the NaN data in `data_year`? Why or why not? 

## 1.7 Aggregating data: A review of groupby-and-summarize
Last week, we introduced the "groupby-and-summarize" operation that is very common in pandas. It's common because we often want to aggregate data by some category. For example, we might want to know the total amount of construction money allocated by school. Or we might want to know the total number of students in each school.

For the projects under construction, let's group by the `Building ID`, which we had a hunch was the same as the `DBN` (Borough and School ID) to get the: 
- Total construction award amount per school

In [ ]:
## Remember that .sum() will only sum the numeric columns
projects_under_const.groupby('Building ID').sum()

Most of these columns are gibberish after we sum (for ex: we don't need a sum of latitudes and longitudes by school). Let's just select the columns we want to use: 

In [ ]:
# Remeber the brackets after a DF allow you to select columns
projects_under_const.groupby('Building ID').sum()['Construction Award']

Let's assign this to a new variable name. 

In [ ]:
projects_under_const_agg = projects_under_const.groupby('Building ID').sum()['Construction Award']

Here you can see that the result is a **pandas Series**. To make this easier to work with during the merge, let's transform this into a pandas DF. 

I'm going to use a function call `.reset_index()` as a trick to do this. `.reset_index()` is a method that resets the index of a dataframe to a column of your choice. The default is to reset the index to a column of sequential numbers

In [ ]:
# See how Building ID, which was the index before, is now a column. 
# and the index is i just 0,...,1180

projects_under_const_agg.reset_index()

In [ ]:
projects_under_const_agg = projects_under_const_agg.reset_index()

In [ ]:
projects_under_const_agg

Let's do something similar with the `class_size` df. As we can see from the below, our data is likely one row per grade and program. We want to aggregate this to the school level. 

In [ ]:
class_size.head()

I'm first going to filter my DF since I just want 'Gen Ed' in order not to skew the representative class size by special programs. 

In [ ]:
# .unique() returns a list of all the unique values in a column
class_size['Program Type'].unique()

In [ ]:
# I am going to use the == operator to check if the value in the Program Type column is equal to 'Gen Ed'
# Then we'll set this filtered dataframe to a new variable
# and use that new dataframe from now on. 
class_size_new = class_size[class_size['Program Type']=='Gen Ed']

In [ ]:
class_size_new.head()

Now, to get the total number of students in each school, I'll have to: 
- Multiply `Number of Classes` and `Number of Students` (let's assume this is per class)
- Sum the total number of students across all classes in a school. 

In [ ]:
class_size_new['total_students_in_grade'] = class_size_new['Number of Students'] * class_size_new['Number of Classes']
# Yes, ignore the SettingWithCopyWarning. 

In [ ]:
class_size_new.head()

Now let's groupby `bid` and sum all the grades within each school. 

In [ ]:
class_size_new.groupby('bid').sum()

Again, we'll just need the `total_students_in_grade` column here. And I'm going to do the `reset_index()` trick again. This time, I'm going to string all these steps together

In [ ]:
# Pandas reads this code from left to right and will apply each function on the right to the everything on the left
# So, first we are going to group by bid
# Then we are going to sum each group
# Then from the entire summed dataframe, we are going to select the total_students_in_grade column
# Selecting that series, we are going to reset the index to create our new dataframe. .

class_size_new_agg = class_size_new.groupby('bid').sum()['total_students_in_grade'].reset_index()

In [ ]:
class_size_new_agg.head()

Finally, we get to do our merge. We are going to merge 
- `projects_under_cont_agg`
- `class_size_new_agg`

In [ ]:
class_size_new_agg.merge(projects_under_const_agg,left_on='bid',right_on='Building ID', how='left')

In [ ]:
merged_df = class_size_new_agg.merge(projects_under_const_agg,left_on='bid',right_on='Building ID', how='left')

Ok, finally, to get to our answer, we're going to apply the `.corr()` function to our dataframe. The [documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html) tells us that this function computes pairwise correlation of columns, excluding NA/null values.

The default method is a 'Pearson' correlation, with all methods being: 
- pearson : standard correlation coefficient
- kendall : Kendall Tau correlation coefficient
- spearman : Spearman rank correlation

In [ ]:
# Yikes, 0.079 correlation. I guess I assumed wrong that there would be an strong correlation between the number of students in a school and the amount of money spent on construction.
merged_df.corr()

In [ ]:
# Using a rank correlation instead of a linear correlation hasn't improved the correlation much.
# (Don't worry about what this is right now.)
merged_df.corr(method = 'spearman')

This was a long lecture, no more exercises!